In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


In [ ]:
# Görüntüyü yükle
input_path = "hela_dataset-train-01/t009.tif"
output_path = "hela_dataset-train-01/t009_processed.tif"

# TIF dosyasını oku
img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

if img is None:
    print(f"Hata: {input_path} dosyası okunamadı!")
else:
    print(f"Orijinal görüntü boyutu: {img.shape}")
    print(f"Orijinal görüntü tipi: {img.dtype}")
    
    # Orijinal görüntüyü göster
    plt.figure(figsize=(8, 8))
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap='gray')
    plt.title('Orijinal Görüntü')
    plt.axis('off')
    plt.show()
    
    # Eğer renkli ise gri tonlamaya çevir
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # 1. Gürültü azaltma - Gaussian blur ile
    denoised = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 2. Binary'ye çevir - Otsu thresholding (otomatik threshold değeri)
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 3. Arka planı siyah yapmak için kontrol et
    # Eğer çoğunluk beyazsa (arka plan beyaz), invert et
    white_pixels = np.sum(binary == 255)
    black_pixels = np.sum(binary == 0)
    
    if white_pixels > black_pixels:
        # Arka plan beyazsa, invert et (arka plan siyah olsun)
        binary = cv2.bitwise_not(binary)
        print("Görüntü invert edildi (arka plan siyah yapıldı)")
    
    print(f"\nBinary görüntü oluşturuldu:")
    print(f"Binary görüntü boyutu: {binary.shape}")
    print(f"Beyaz pikseller: {np.sum(binary == 255)}, Siyah pikseller: {np.sum(binary == 0)}")
    
    # Binary görüntüyü göster
    plt.figure(figsize=(8, 8))
    plt.imshow(binary, cmap='gray')
    plt.title('Binary Görüntü (Noise Kaldırılmış)')
    plt.axis('off')
    plt.show()


In [ ]:
# Morphological işlemler için kernel oluştur
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
# Alternatif kernel seçenekleri:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))

# 1. Closing işlemi: Dilation + Erosion
# Küçük delikleri kapatır ve nesneleri birleştirir
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
print(f"Closing işlemi tamamlandı")
print(f"Closing öncesi - Beyaz: {np.sum(binary == 255)}, Siyah: {np.sum(binary == 0)}")
print(f"Closing sonrası - Beyaz: {np.sum(closed == 255)}, Siyah: {np.sum(closed == 0)}")

# Closing sonucunu göster
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(binary, cmap='gray')
plt.title('Closing Öncesi')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(closed, cmap='gray')
plt.title('Closing Sonrası')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# 2. Opening işlemi: Erosion + Dilation
# Küçük gürültüleri ve ince çıkıntıları temizler
opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)
print(f"Opening işlemi tamamlandı")
print(f"Opening sonrası - Beyaz: {np.sum(opened == 255)}, Siyah: {np.sum(opened == 0)}")

# Opening sonucunu göster
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(closed, cmap='gray')
plt.title('Opening Öncesi (Closing Sonrası)')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(opened, cmap='gray')
plt.title('Opening Sonrası')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Connected Components Analizi
# Bileşenleri analiz et (stats ile alan bilgisi al)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened, connectivity=8)

print(f"\nConnected Components Analizi:")
print(f"Toplam bileşen sayısı (arka plan dahil): {num_labels}")
print(f"Gerçek nesne sayısı: {num_labels - 1}")  # 0 = arka plan

# Küçük bileşenleri silmek için eşik değeri (piksel sayısı)
min_pixel_size = 100  # Bu değeri ihtiyaca göre ayarlayabilirsiniz

# Yeni görüntü oluştur (sadece büyük bileşenler kalacak)
filtered = np.zeros_like(opened)

# Her bileşeni kontrol et (0 = arka plan, atla)
removed_count = 0
kept_count = 0

for label_id in range(1, num_labels):  # 0'ı atla (arka plan)
    area = stats[label_id, cv2.CC_STAT_AREA]
    
    if area >= min_pixel_size:
        # Büyük bileşeni koru
        filtered[labels == label_id] = 255
        kept_count += 1
    else:
        # Küçük bileşeni sil
        removed_count += 1

print(f"\nKüçük bileşen temizleme:")
print(f"Eşik değeri: {min_pixel_size} piksel")
print(f"Korunan bileşen sayısı: {kept_count}")
print(f"Silinen bileşen sayısı: {removed_count}")
print(f"Filtreleme öncesi - Beyaz: {np.sum(opened == 255)}, Siyah: {np.sum(opened == 0)}")
print(f"Filtreleme sonrası - Beyaz: {np.sum(filtered == 255)}, Siyah: {np.sum(filtered == 0)}")

# Sonucu kaydet
cv2.imwrite(output_path, filtered)
print(f"\nKüçük bileşenler temizlenmiş görüntü kaydedildi: {output_path}")

# Sonuçları göster
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(opened, cmap='gray')
plt.title('Filtreleme Öncesi\n(Closing + Opening)')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(filtered, cmap='gray')
plt.title(f'Filtreleme Sonrası\n(Min: {min_pixel_size} piksel)')
plt.axis('off')
plt.subplot(1, 3, 3)
# Fark görüntüsü (silinen componentları göster)
diff = opened.copy()
diff[filtered == 255] = 0  # Korunanları siyah yap
plt.imshow(diff, cmap='hot')
plt.title('Silinen Componentlar')
plt.axis('off')
plt.tight_layout()
plt.show()

# Final sonucu göster
plt.figure(figsize=(8, 8))
plt.imshow(filtered, cmap='gray')
plt.title(f'Final Sonuç\n({kept_count} bileşen korundu, {removed_count} bileşen silindi)')
plt.axis('off')
plt.show()


In [ ]:
# Opening işlemi: Erosion + Dilation
# Küçük gürültüleri ve ince çıkıntıları temizler
opened_final = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
print(f"Opening işlemi tamamlandı")
print(f"Opening öncesi - Beyaz: {np.sum(filtered == 255)}, Siyah: {np.sum(filtered == 0)}")
print(f"Opening sonrası - Beyaz: {np.sum(opened_final == 255)}, Siyah: {np.sum(opened_final == 0)}")

# Opening sonucunu göster
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(filtered, cmap='gray')
plt.title('Opening Öncesi\n(Connected Components Filtreleme Sonrası)')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(opened_final, cmap='gray')
plt.title('Opening Sonrası')
plt.axis('off')
plt.tight_layout()
plt.show()

# Final sonucu kaydet
cv2.imwrite(output_path, opened_final)
print(f"\nFinal görüntü kaydedildi: {output_path}")

# Final sonucu göster
plt.figure(figsize=(8, 8))
plt.imshow(opened_final, cmap='gray')
plt.title('Final Sonuç (Opening Uygulanmış)')
plt.axis('off')
plt.show()


In [ ]:
# Sadece içi kapalı (deliksiz) şekilleri bırak
# Contour hierarchy kullanarak delikli şekilleri tespit et

# Önce filtered görüntüsünü kullan (eğer yoksa opened_final kullan)
input_img = filtered if 'filtered' in globals() else opened_final

# Contour'ları bul (hierarchy ile)
contours, hierarchy = cv2.findContours(input_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

print(f"Toplam contour sayısı: {len(contours)}")

# İçi kapalı şekilleri tutmak için yeni görüntü
solid_only = np.zeros_like(input_img)

# Her contour'u kontrol et
solid_count = 0
hollow_count = 0

if hierarchy is not None:
    for i, contour in enumerate(contours):
        # Hierarchy yapısı: [next, previous, first_child, parent]
        # Eğer first_child == -1 ise, iç contour yok (içi kapalı)
        # Eğer first_child != -1 ise, iç contour var (delik var)
        
        has_hole = False
        if hierarchy[0][i][2] != -1:  # first_child var mı?
            has_hole = True
        
        # Contour alanını kontrol et (çok küçük contour'ları atla)
        area = cv2.contourArea(contour)
        if area < 50:  # Çok küçük contour'ları atla
            continue
        
        if not has_hole:
            # İçi kapalı şekil - koru
            cv2.drawContours(solid_only, [contour], -1, 255, -1)  # -1 = içini doldur
            solid_count += 1
        else:
            # Delikli şekil - atla
            hollow_count += 1

print(f"\nİçi kapalı şekil analizi:")
print(f"İçi kapalı (solid) şekil sayısı: {solid_count}")
print(f"Delikli (hollow) şekil sayısı: {hollow_count}")
print(f"Filtreleme öncesi - Beyaz: {np.sum(input_img == 255)}, Siyah: {np.sum(input_img == 0)}")
print(f"Filtreleme sonrası - Beyaz: {np.sum(solid_only == 255)}, Siyah: {np.sum(solid_only == 0)}")

# Sonucu kaydet
cv2.imwrite(output_path, solid_only)
print(f"\nSadece içi kapalı şekiller kaydedildi: {output_path}")

# Sonuçları göster
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(input_img, cmap='gray')
plt.title('Filtreleme Öncesi\n(Tüm Şekiller)')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(solid_only, cmap='gray')
plt.title(f'Filtreleme Sonrası\n(Sadece İçi Kapalı: {solid_count})')
plt.axis('off')
plt.subplot(1, 3, 3)
# Fark görüntüsü (silinen delikli şekilleri göster)
diff = input_img.copy()
diff[solid_only == 255] = 0  # Korunanları siyah yap
plt.imshow(diff, cmap='hot')
plt.title(f'Silinen Delikli Şekiller\n({hollow_count} adet)')
plt.axis('off')
plt.tight_layout()
plt.show()

# Final sonucu göster
plt.figure(figsize=(8, 8))
plt.imshow(solid_only, cmap='gray')
plt.title(f'Sadece İçi Kapalı Şekiller\n({solid_count} solid, {hollow_count} hollow silindi)')
plt.axis('off')
plt.show()


In [ ]:
# Connected Components Analizi ile küçük componentları sil
# Önceki hücreden solid_only görüntüsünü kullan
input_img = solid_only if 'solid_only' in globals() else filtered

# Connected Components Analizi
# Bileşenleri analiz et (stats ile alan bilgisi al)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(input_img, connectivity=8)

print(f"\nConnected Components Analizi:")
print(f"Toplam bileşen sayısı (arka plan dahil): {num_labels}")
print(f"Gerçek nesne sayısı: {num_labels - 1}")  # 0 = arka plan

# Küçük bileşenleri silmek için eşik değeri (piksel sayısı)
min_pixel_size = 2000  # Bu değeri ihtiyaca göre ayarlayabilirsiniz

# Yeni görüntü oluştur (sadece büyük bileşenler kalacak)
filtered_final = np.zeros_like(input_img)

# Her bileşeni kontrol et (0 = arka plan, atla)
removed_count = 0
kept_count = 0

for label_id in range(1, num_labels):  # 0'ı atla (arka plan)
    area = stats[label_id, cv2.CC_STAT_AREA]
    
    if area >= min_pixel_size:
        # Büyük bileşeni koru
        filtered_final[labels == label_id] = 255
        kept_count += 1
    else:
        # Küçük bileşeni sil
        removed_count += 1

print(f"\nKüçük bileşen temizleme:")
print(f"Eşik değeri: {min_pixel_size} piksel")
print(f"Korunan bileşen sayısı: {kept_count}")
print(f"Silinen bileşen sayısı: {removed_count}")
print(f"Filtreleme öncesi - Beyaz: {np.sum(input_img == 255)}, Siyah: {np.sum(input_img == 0)}")
print(f"Filtreleme sonrası - Beyaz: {np.sum(filtered_final == 255)}, Siyah: {np.sum(filtered_final == 0)}")

# Sonucu kaydet
cv2.imwrite(output_path, filtered_final)
print(f"\nKüçük bileşenler temizlenmiş görüntü kaydedildi: {output_path}")

# Sonuçları göster
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(input_img, cmap='gray')
plt.title('Filtreleme Öncesi\n(İçi Kapalı Şekiller)')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(filtered_final, cmap='gray')
plt.title(f'Filtreleme Sonrası\n(Min: {min_pixel_size} piksel)')
plt.axis('off')
plt.subplot(1, 3, 3)
# Fark görüntüsü (silinen componentları göster)
diff = input_img.copy()
diff[filtered_final == 255] = 0  # Korunanları siyah yap
plt.imshow(diff, cmap='hot')
plt.title('Silinen Küçük Componentlar')
plt.axis('off')
plt.tight_layout()
plt.show()

# Final sonucu göster
plt.figure(figsize=(8, 8))
plt.imshow(filtered_final, cmap='gray')
plt.title(f'Final Sonuç\n({kept_count} bileşen korundu, {removed_count} bileşen silindi)')
plt.axis('off')
plt.show()
